In [ ]:
import numpy as np
import xarray as xr
import hvplot.xarray
import param
import holoviews as hv; hv.extension('bokeh')
from holoviews.streams import Stream
import panel as pn; pn.extension()

In [ ]:
n_frames = 300
height = 256
width = 256

# Create random float values between 0 and 1, then scale to between 0 and 255
noise = np.random.rand(n_frames, height, width) * 256

# Convert to unsigned 8-bit integer (this rounds down so max is 255)
noise = noise.astype(np.uint8)

data = xr.DataArray(
    noise,
    coords={"frame": range(n_frames), "height": range(height), "width": range(width)},
    dims=["frame", "height", "width"],
)

In [ ]:
data

## Viewer with default slider

In [ ]:
data.hvplot.image(groupby="frame", cmap="Viridis", frame_height=400, frame_width=400, colorbar=False) 

## Viewer with Player widget
- Task: is there a simpler or more performant implementation of this?

In [ ]:
frames = data.coords["frame"].values
f_min = int(frames.min())
f_max = int(frames.max())
height = data.sizes["height"]
width = data.sizes["width"]

# Generate Image object for a given frame
def generate_image(frame, data):  
    return hv.Image(
        data.sel(frame=frame).compute(), kdims=["width", "height"]
    )

# Setup frame stream
frame_param = param.Integer(default=f_min, bounds=(f_min, f_max))
FrameStream = Stream.define("FrameStream", frame=frame_param)
frame_stream = FrameStream()

# Dynamic map of image via frame stream
image_generator = pn.bind(generate_image, data=data)
image_map = hv.DynamicMap(image_generator, streams=[frame_stream]).opts(
    frame_width=400, aspect=width / height, cmap="Viridis"
)

# Create a video player widget
video_player = pn.widgets.Player(
    length=len(frames), interval=10, value=f_min, width=600, height=90
)

# update the frame stream when a new event occurs on the widget
def update_frame_stream(event):
    frame_stream.event(frame=int(frames[event.new]))

# Link player widget to the frame stream update function
video_player.param.watch(update_frame_stream, "value")

layout = pn.layout.Column(video_player, image_map)
layout